In [2]:
import lipd
import pandas as pd
import os
from collections import defaultdict

In [3]:
input_path = 'D:\\annotating_paleoclimate_data\\recommender\\download_files\\linked_earth_wiki_files'

tup = list(os.walk(input_path))
root, dir_names, file_names = tup[0][0], tup[0][1], tup[0][2]

In [4]:
table = pd.DataFrame(columns = ['filename','archiveType', 'inferredVariableType','units','proxyObservationType','interpretation/variable','interpretation/variableDetail'])

In [5]:
author_info = set()
investigator_info = set()
no_info = set()
dataCon = set()
reverse_index_author = {}

In [ ]:
def add_to_reversed_index_author_map(val, archive):
    for a_name in val.split(','):
        reverse_index_author.setdefault(a_name,[]).append(archive)

In [ ]:
for line in file_names:
    print(line)
    if '.lpd' in line:
        filen = line.strip()
        l = os.path.join(root, line.strip())
        d= lipd.readLipd(l)
        if 'archiveType' not in d:
            continue
        archive = d['archiveType']
        if 'pub' in d:
            path = d['pub'][0]
            if 'author' in path:
                print(path['author'])
                author_list = path['author']
                for auth in author_list:
                    if 'name' in auth:
                        val = auth['name'][0] if type(auth['name']) == list else auth['name']
                        add_to_reversed_index_author_map(val, archive)
                author_info.add(line)
        if line not in author_info and 'investigator' in d:
            path_investigators = d['investigator']
            print(path_investigators)
            for auth in author_list:
                if 'name' in auth:
                    val = auth['name'][0] if type(auth['name']) == list else auth['name']
                    add_to_reversed_index_author_map(val, archive)
            investigator_info.add(line)
        elif line not in author_info and line not in investigator_info :
            no_info.add(line)

In [ ]:
import collections
for key in reverse_index_author.keys():
    reverse_index_author[key] = dict(collections.Counter(reverse_index_author[key]))

In [ ]:
reverse_index_author['Y. Shibata'] = reverse_index_author['and Y. Shibata. Harada']
del reverse_index_author['and Y. Shibata. Harada']

reverse_index_author['N. Harada'] = reverse_index_author[' N.']
del reverse_index_author[' N.']

reverse_index_author['N. Ahagon'] = reverse_index_author[' N. Ahagon']
del reverse_index_author[' N. Ahagon']

reverse_index_author['T. Sakamoto'] = reverse_index_author[' T. Sakamoto']
del reverse_index_author[' T. Sakamoto']

reverse_index_author['M. Uchida'] = reverse_index_author[' M. Uchida']
del reverse_index_author[' M. Uchida']

reverse_index_author[' M. Ikehara'] = reverse_index_author[' M. Ikehara']
del reverse_index_author[' M. Ikehara']

reverse_index_author['W.L. Prell'] = reverse_index_author['W.L. Fairbanks']
del reverse_index_author['W.L. Fairbanks']

reverse_index_author['R.G. Fairbanks'] = reverse_index_author[' R.G.']
del reverse_index_author[' R.G.']
del reverse_index_author[' Prell']

In [ ]:
author_names = reverse_index_author.keys()

In [ ]:
seed_author_list = {}
for n in author_names:
    n = n.title()
    n_list = n.split()
    with_ini = False
    for ind, inner_n in enumerate(n_list):
        if ind != 1 and '.' in inner_n:
            with_ini = True            
    if not with_ini:
        try:
            seed_author_list[n[0].upper()].add(n)
        except KeyError:
            seed_author_list[n[0].upper()] = {n}

In [ ]:
seed_author_list['K'].remove('Karl-Heinz \xa0Baumann')
seed_author_list['N'].remove('Nicholas J Shackleton')
seed_author_list['N'].remove('Nicholas Shackleton')
seed_author_list['N'].remove('Nick Shackleton')
seed_author_list['P'].remove('Peter J Müller')
seed_author_list['R'].remove('Ralph R Schneider')
seed_author_list['R'].remove('Ralph Schneider')

seed_author_list['R'].add('R.G. Fairbanks')
seed_author_list['W'].add('W.L. Prell')

In [ ]:
com_same = []
last_same = []
ref_same = []
iter_same = []

In [ ]:
def editDistance(str1, str2, m, n):
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]
 
    # Fill d[][] in bottom up manner
    for i in range(m + 1):
        for j in range(n + 1):
 
            if i == 0:
                dp[i][j] = j    # Min. operations = j
 
            elif j == 0:
                dp[i][j] = i    # Min. operations = i
 
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]

            else:
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert
                                   dp[i-1][j],        # Remove
                                   dp[i-1][j-1])    # Replace
 
    return dp[m][n]

In [ ]:
def check_fname(auth_list, rn_list):
    if ('.' in auth_list[0] and rn_list[0][0] == auth_list[0][0]):
        return True
    elif rn_list[0] == auth_list[0]:
        return True
    else:
        matching = 0
        auth_fname, rn_fname = auth_list[0], rn_list[0]
        ap, rp = 0, 0
        while ap < len(auth_fname) and rp < len(rn_fname):
            if auth_fname[ap] == rn_fname[rp]:
                matching += 1
                ap += 1
                rp += 1
            else:
                break
        if matching >= 2:
            return True
        return False

In [ ]:
author_names_map = {}
for n in author_names:
    auth = n.title()
    
    reference_names = seed_author_list[auth[0].upper()]
    for rn in reference_names:

        auth_list, rn_list = auth.split(), rn.split()
        auth_lastName, rn_lastName = auth_list[-1], rn_list[-1]
        # complete names are same
        if rn_list == auth_list:
            author_names_map[n] = rn
            com_same.append(n)
            break
        # lastNames are completely equal
        elif rn_lastName == auth_lastName:
            if check_fname(auth_list, rn_list):
                author_names_map[n] = rn
                last_same.append(n)
                break
        # refernce lastName is a substring of input lastName
        elif rn_lastName in auth_lastName:
            if check_fname(auth_list, rn_list):
                author_names_map[n] = rn
                ref_same.append(n)
                break
        # check where the characters of the lastName are different
        if len(auth_lastName) >= 3 and len(rn_lastName) >= 3:     
            edit_dist = editDistance(auth_lastName, rn_lastName, len(auth_lastName), len(rn_lastName))
            if edit_dist <= 2:
                if check_fname(auth_list, rn_list):
                    author_names_map[n] = rn
                    iter_same.append(n)
                    break
    if n not in author_names_map:
        author_names_map[n] = auth        
        

In [ ]:
# change to FirstName. LastName format
        
author_names_map['SARNTHEIN M.'] = 'M. Sarthein'
author_names_map['ERLENKEUSER H.'] = 'H. Erlenkeuser'
author_names_map['KUCERA M.'] = 'M. Kucera'
author_names_map['PFLAUMANN U.'] = 'U. Pflaumann'
author_names_map['SCHULZ M.'] = 'M. Schulz'

In [6]:
table1 = pd.DataFrame(columns = ['filename','archiveType','proxyObservationType','units'])
table2 = pd.DataFrame(columns = ['filename','archiveType','proxyObservationType', 'inferredVariableType','units'])

ignore_inf_types = ['Age', 'Year']

In [7]:
seq1, seq2 = 0, 0
uniPr, proPr = 0, 0

In [8]:
input_path = 'D:\\annotating_paleoclimate_data\\recommender\\download_files\\linked_earth_wiki_files_new'
# input_path = 'D:\\annotating_paleoclimate_data\\recommender\\download_files\\temp_folder'

tup = list(os.walk(input_path))
root, dir_names, file_names = tup[0][0], tup[0][1], tup[0][2]


In [9]:
for line in file_names:
    print(line)
    if '.lpd' in line:
        filen = line.strip()
        line = os.path.join(root, line.strip())
        d = lipd.readLipd(line)
        if 'paleoData' not in d:
            continue
        path = d['paleoData']['paleo0']['measurementTable']['paleo0measurement0']['columns']
#         print(path.keys())
        archive = d['archiveType']
        # print('archive: ', archive)
        for key in path.keys() :
            print(path[key].keys())
            
            unit = 'NotApplicable'
            inferredVType = 'NA'
            proxyOType = 'NA'
            if 'proxyObservationType' in path[key].keys() :
                if 'units' in path[key].keys() :
                    unit = path[key]['units']
                    uniPr += 1
                proxyOType = path[key]['proxyObservationType']
                
                if proxyOType != 'NA':
                    df = pd.DataFrame({'filename':[filen], 'archiveType': [archive],'proxyObservationType':[proxyOType],'units':[unit]})
                    table1 = table1.append(df, ignore_index = True)
                    seq1 += 1
                
            elif 'inferredVariableType' in path[key].keys() :
                if 'units' in path[key].keys() :
                    unit = path[key]['units']
                if 'inferredFrom' in path[key].keys() :
                    if 'proxyObservationType' in path[key]['inferredFrom']:
                        proxyOType = path[key]['inferredFrom']['proxyObservationType']
                        proPr += 1
                inferredVType = path[key]['inferredVariableType']
            
                if inferredVType not in ignore_inf_types and (inferredVType != 'NA' or proxyOType != 'NA'): 
                    df = pd.DataFrame({'filename':[filen], 'archiveType': [archive],'proxyObservationType':[proxyOType],'inferredVariableType': [inferredVType],'units':[unit]})
                    table2 = table2.append(df, ignore_index = True)
                    seq2 += 1

A7.Oppo.2005.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: A7.Oppo.2005.lpd
Finished read: 1 record
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'variableName', 'variableType', 'values', 'hasResolution'])
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'interpretation', 'variableName', 'takenAtDepth', 'variableType', 'values', 'hasResolution'])
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'interpretation', 'variableName', 'takenAtDepth', 'variableType', 'values', 'hasResolution'])
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'interpretation', 'instrument', 'measurementMethod', 'variableName', 'proxyObservationType', 'measurementStandard', 'takenAtDepth', 'variableType', 'value

reading: Ant-DomeC.Stenni.2001.lpd
Finished read: 1 record
dict_keys(['dataType', 'detail', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'measurementMaterial', 'measurementMethod', 'variableName', 'notes', 'pages2kID', 'proxyObservationType', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'variableType', 'values', 'hasResolution'])
Ant-DomeF1993.Uemura.2014.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: Ant-DomeF1993.Uemura.2014.lpd
Finished read: 1 record
dict_keys(['dataType', 'detail', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'measurementMaterial', 'measurementMethod', 'variableName', 'notes', 'pages2kID', 'proxyObservationType', 'qC

Finished read: 1 record
dict_keys(['dataType', 'detail', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'measurementMaterial', 'measurementMethod', 'variableName', 'notes', 'pages2kID', 'proxyObservationType', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'variableType', 'values', 'hasResolution'])
Ant-US-ITASE-2000-1.Steig.2013.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: Ant-US-ITASE-2000-1.Steig.2013.lpd
Finished read: 1 record
dict_keys(['dataType', 'detail', 'number', 'h

Finished read: 1 record
dict_keys(['dataType', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'measurementMaterial', 'variableName', 'notes', 'pages2kID', 'proxyObservationType', 'qCCertification', 'qCnotes', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'variableType', 'values', 'hasResolution'])
Arc-Austfonna.Isaksson.2005.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: Arc-Austfonna.Isaksson.2005.lpd
Error: Unable to update file to LiPD v1.3: dictionary keys changed during iteration
Error: Unable to update file to LiPD v1.3: dictionary keys changed during iteration
Finished read: 1 record
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'variableName'

Error: Unable to update file to LiPD v1.3: dictionary keys changed during iteration
Error: Unable to update file to LiPD v1.3: dictionary keys changed during iteration
Finished read: 1 record
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'variableName', 'notes', 'pages2kID', 'proxyObservationType', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'variableType', 'values', 'hasResolution'])
Arc-GulfofAlaska.Wilson.2014.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: Arc-GulfofAlaska.Wilson.2014.lpd
Finished read: 1 record
dict_keys(['calibration', 'number', 'TSid', 'inferredVariableType', 'variableName', 'proxyObservationType', 'useInGlobalTemperatureAnalysis', 'variableType', 'v

...

Error: lipd_read: unable to read LiPD: File is not a zip file
Finished read: 1 record
Ocn-TagusMudPatch.Abrantes.2005.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: Ocn-TagusMudPatch.Abrantes.2005.lpd
Finished read: 1 record
dict_keys(['dataType', 'number', 'TSid', 'variableName', 'notes', 'proxyObservationType', 'qCnotes', 'takenAtDepth', 'useInGlobalTemperatureAnalysis', 'variableType', 'values'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'takenAtDepth', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'units', 'TSid', 'variableName', 'variableType', 'values', 'hasResolution', 'hasMinValue'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferred

Finished read: 1 record
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'iso2kUI', 'variableName', 'notes', 'ocean2kID', 'pages2kID', 'proxyObservationType', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'variableType', 'values', 'hasResolution'])
Ocn-Vanuatu.Gorman.2012.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: Ocn-Vanuatu.Gorman.2012.lpd
Finished read: 1 record
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'iso2kUI', 'variableName', 'notes', 'ocean2kID', 'pages2kID', 'proxyObservationType', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'descr

dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'instrument', 'measurementMethod', 'variableName', 'proxyObservationType', 'measurementStandard', 'takenAtDepth', 'variableType', 'values', 'hasResolution'])
ODP659.Tiedemann.1994.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: ODP659.Tiedemann.1994.lpd
Finished read: 1 record
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'variableName', 'variableType', 'values', 'hasResolution'])
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'variableName', 'notes', 'proxyObservationType', 'takenAtDepth', 'variableType', 'values', 'hasResolution'])
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'variableName', 'notes', 'proxyObservationType', 'takenAtDepth', 'variableType', 'values', 'hasResolution

reading: SAm-LagunaAculeo.vonGunten.2009.lpd
Finished read: 1 record
dict_keys(['calibration', 'dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'notes', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'variableType', 'values', 'hasResolution'])
dict_keys(['calibration', 'dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVariableType', 'variableName', 'notes', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution'])
dict_keys(['calibration', 'dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'T

dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'hasMinValue', 'units', 'TSid', 'variableName', 'notes', 'proxyObservationType', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution', 'hasMedianValue'])
dict_keys(['dataType', 'description', 'number', 'hasMaxValue', 'hasMeanValue', 'units', 'TSid', 'variableName', 'notes', 'proxyObservationType', 'qCCertification', 'useInGlobalTemperatureAnalysis', 'variableType', 'values', 'hasResolution', 'hasMinValue', 'hasMedianValue'])
SK168-GC1.Gebregiorgis.2016.lpd
Disclaimer: LiPD files may be updated and modified to adhere to standards

reading: SK168-GC1.Gebregiorgis.2016.lpd
Finished read: 1 record
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'variableName', 'variableType', 'values', 'hasResolution'])
dict_keys(['number', 'hasMaxValue', 'hasMeanValue', 'hasMedianValue', 'hasMinValue', 'units', 'TSid', 'inferredVaria

In [11]:
table_com=table1.explode('proxyObservationType').explode('units').reset_index()
table_com = table_com.drop(columns = ['index'])
table_com.to_csv('arch_pro_uni.csv', sep = ',', encoding = 'utf-8',index = False)

table_com=table2.explode('proxyObservationType').explode('units').explode('inferredVariableType').reset_index()
table_com = table_com.drop(columns = ['index'])
table_com.to_csv('arch_pro_inf_uni.csv', sep = ',', encoding = 'utf-8',index = False)

In [12]:
print('seq1' , seq1)
print('seq2' , seq2)

seq1 2230
seq2 175


In [13]:
print('uniPresent' , uniPr)
print('proPr' , proPr)

uniPresent 2136
proPr 91
